In [2]:
pip install fuzzywuzzy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd
from fuzzywuzzy import process

# Load the CSV files
i_m_h_streets = pd.read_csv('i_m_h_streets.csv', header=None, names=['i_m_h_street_name'])
ta_streets = pd.read_csv('ta_streets.csv', header=None, names=['ta_street'])

# Normalize the street names and convert NaNs to empty strings
i_m_h_streets['i_m_h_street_name'] = i_m_h_streets['i_m_h_street_name'].fillna('').str.lower().str.strip()
ta_streets['ta_street'] = ta_streets['ta_street'].fillna('').str.lower().str.strip()

# Function to find the best match
def find_best_match(street_name, choices, threshold=80):
    result = process.extractOne(street_name, choices)
    if result:
        match, score = result[0], result[1]
        if score >= threshold:
            return match
    return None

# Create the result DataFrame
result = pd.DataFrame(columns=['i_m_h_street_name', 'ta_street', 'result_name'])

for index, row in i_m_h_streets.iterrows():
    i_m_h_name = row['i_m_h_street_name']
    best_match = find_best_match(i_m_h_name, ta_streets['ta_street'])
    
    if best_match:
        result_name = i_m_h_name
    else:
        result_name = i_m_h_name if i_m_h_name else 'no_match'
    
    # Use pd.concat instead of append
    result = pd.concat([result, pd.DataFrame([{'i_m_h_street_name': i_m_h_name, 'ta_street': best_match, 'result_name': result_name}])], ignore_index=True)

# Handle streets in ta_streets not matched in i_m_h_streets
for index, row in ta_streets.iterrows():
    ta_name = row['ta_street']
    if ta_name not in result['ta_street'].values:
        result = pd.concat([result, pd.DataFrame([{'i_m_h_street_name': None, 'ta_street': ta_name, 'result_name': ta_name}])], ignore_index=True)

# Save the result to a new CSV file
result.to_csv('matched_streets.csv', index=False)


In [12]:
result

,i_m_h_street_name,ta_street,result_name
0,0,440,0
1,,None,no_match
2,אב,אב,אב
3,אבא ושרה נאמן,נאמן אבא ושרה,אבא ושרה נאמן
4,אבודרהם,אבודרהם,אבודרהם
...,...,...,...
2744,None,ויל קורט,ויל קורט
2745,None,1362,1362
2746,None,470,470
2747,None,3969,3969


In [10]:
i_m_h_name

nan

In [5]:
i_m_h_streets

,i_m_h_street_name
0,0
1,NaN
2,אב
3,אבא ושרה נאמן
4,אבודרהם
...,...
1670,"תרצ""ו"
1671,תרצה
1672,"תש""ח"
1673,תשבי


In [6]:

# Normalize the street names
i_m_h_streets['i_m_h_street_name'] = i_m_h_streets['i_m_h_street_name'].str.lower().str.strip()
ta_streets['ta_street'] = ta_streets['ta_street'].str.lower().str.strip()

# Function to find the best match
def find_best_match(street_name, choices, threshold=80):
    match, score = process.extractOne(street_name, choices)
    if score >= threshold:
        return match
    return None

# Create the result DataFrame
result = pd.DataFrame(columns=['i_m_h_street_name', 'ta_street', 'result_name'])

for index, row in i_m_h_streets.iterrows():
    i_m_h_name = row['i_m_h_street_name']
    best_match = find_best_match(i_m_h_name, ta_streets['ta_street'])
    
    if best_match:
        result_name = i_m_h_name
    else:
        result_name = i_m_h_name if i_m_h_name else 'no_match'
    
    result = result.append({'i_m_h_street_name': i_m_h_name, 'ta_street': best_match, 'result_name': result_name}, ignore_index=True)

# Handle streets in ta_streets not matched in i_m_h_streets
for index, row in ta_streets.iterrows():
    ta_name = row['ta_street']
    if ta_name not in result['ta_street'].values:
        result = result.append({'i_m_h_street_name': None, 'ta_street': ta_name, 'result_name': ta_name}, ignore_index=True)

# Save the result to a new CSV file
result.to_csv('matched_streets.csv', index=False)


ValueError: too many values to unpack (expected 2)